## The Advanced Computing Center for Research and Education (ACCRE) is a computer cluster serving the high-performance computing needs of research for Vanderbilt University. 

**In this data question, we will be analyzing data on jobs run on ACCRE's hardware**

In [1]:
import pandas as pd

**DATA DICTIONAR**
**JOBID**	The identification number of the job or job step. Regular jobs are in the form JobID and array jobs are in the form ArrayJobID_ArrayTaskID;
**ACCOUNT**	The account under which the job was submitted;
**USER**	The user who submitted the job;
**USEDMEM**	Used memory in megabytes per-node
**REQTIME**	Requested time in d-hh:mm:ss or hh:mm:ss;
**USEDTIME**	Used time in d-hh:mm:ss or hh:mm:ss;
**NODES**	Number of servers used for this job;
**CPUS**	Total number of CPU-cores allocated to the job;
**GPUS**	Total number of GPUs allocated to the job;
**PARTITION**	Identifies the partition on which the job ran;
**EXITCODE**	The exit code returned by the job script or salloc, typically as set by the exit() function. Following the colon is the signal that caused the process to terminate if it was terminated by a signal;
**STATE**	Job state or status (COMPLETED, CANCELLED, FAILED, TIMEOUT, PREEMPTED, etc.);

In [2]:
df1 = pd.read_csv('data/accre-gpu-jobs-2022-v2.csv', error_bad_lines=False)

C:\Users\upadh\AppData\Local\Temp\ipykernel_16564\3937202199.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv('data/accre-gpu-jobs-2022-v2.csv', error_bad_lines=False)
b'Skipping line 124084: expected 12 fields, saw 13\nSkipping line 124809: expected 12 fields, saw 13\n'


In [3]:
df1.head()

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED


In [4]:
#(mem  *  usedtime)/usedtime

In [5]:
df1.shape

(336950, 12)

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336950 entries, 0 to 336949
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   JOBID      336950 non-null  object
 1   ACCOUNT    336950 non-null  object
 2   USER       336950 non-null  object
 3   USEDMEM    336950 non-null  object
 4   REQTIME    336950 non-null  object
 5   USEDTIME   336950 non-null  object
 6   NODES      336950 non-null  int64 
 7   CPUS       336950 non-null  int64 
 8   GPUS       336950 non-null  int64 
 9   PARTITION  336950 non-null  object
 10  EXITCODE   336950 non-null  object
 11  STATE      336950 non-null  object
dtypes: int64(3), object(9)
memory usage: 30.8+ MB


**DATA Questions:**
    The main objectives of this project are to examine the following questions:

Q1 What is the distribution of per-GPU main memory usage over all runtime-weighed jobs in each partition? Knowing this will help ACCRE to understand our users memory needs for future hardware purchases.

Q2 What is the distribution of the number of GPUs in each job (runtime-weighted) for each partition? What fraction of runtime-weighted and GPU-weighted jobs are using more than 4 GPUs and thus probably using the RoCE networking? Is this fraction different for each partition?

Q3 What is the total runtime usage per-gpu (i.e. multiply runtime by the number of gpus) in each of the 3 partitions over the last year?

Q4 What is the distribution of different groups and users accessing each partition? In each partition, who are the top users, and do they represent a majority of the runtime-weighted jobs on the partition?

Q5 Currently there is a 5 day limit on runtime for GPU jobs, although some users have been asking for extensions. What is the distribution of requested runtime and actual runtime on jobs on each partition? Do users really need more time, or are they simply always requesting the maximum?

In [7]:
df1['ACCOUNT'].nunique()

27

In [8]:
df1['ACCOUNT'].value_counts()

glasshouse       247125
malanga           51677
virginia          10342
prince             7499
grisette           5828
harovin            5046
antares            3222
sunburst           1522
pasilla            1362
plumcot            1080
pecans              628
kinnow              436
bertanne            221
almonds             221
apple               210
cactus              117
clara               100
orange               79
rosemarie            56
horseradish          52
sugranineteen        43
white                36
pinole               20
gem                  19
packham               7
bunch                 1
summer                1
Name: ACCOUNT, dtype: int64

In [9]:
df1['USEDTIME'].value_counts()

00:00:00    51083
00:00:07    19788
00:00:08    11147
00:00:09     6900
00:00:06     5785
            ...  
09:02:07        1
09:41:15        1
09:04:46        1
05:20:19        1
01:37:38        1
Name: USEDTIME, Length: 20818, dtype: int64

In [10]:
df1['USEDMEM'].value_counts()

0          58747
5.53M      27838
7.41M      17001
5.54M      13246
7.42M       3473
           ...  
214.51M        1
219.42M        1
217.21M        1
217.34M        1
145.24M        1
Name: USEDMEM, Length: 114062, dtype: int64

In [11]:
df1['REQTIME'].value_counts()

10:00:00       138155
2-00:00:00      54776
00:30:00        30667
5-00:00:00      28541
12:00:00        14080
                ...  
13:00:00            1
10-00:00:00         1
2-05:00:00          1
00:22:00            1
11-00:00:00         1
Name: REQTIME, Length: 76, dtype: int64

In [12]:
df1.groupby(['ACCOUNT','PARTITION'])['USEDMEM'].sum().sort_index()

ACCOUNT        PARTITION
almonds        turing       43427.70M040875.91M40023.96M44535.52M43227.90M...
antares        pascal       1475.62M1779.30M1784.97M1784.22M1780.81M1783.6...
               turing       3891.07M3901.01M3804.57M3816.78M3991.31M4022.3...
apple          maxwell                                                 16.76M
               turing       1439.46M149.39M35636.29M23944.04M35463.89M3797...
bertanne       maxwell      007.41M7.41M7.41M9.22M9.22M7.42M6.02M7.36M7.41...
               pascal       0006.05M123.93M125.50M000136.18M434.74M1335.41...
               turing       05.43M7.43M7.42M5.41M5.43M5.41M5.41M5.43M7.43M...
bunch          turing                                                   6.09M
cactus         turing       01341.45M0266.16M271.75M0140.36M262.86M143.54M...
clara          maxwell      549.10M638.61M000000006.16M0418.48M0840.29M000...
gem            turing       7.85M7.41M9.21M3587.85M3623.39M12139.35M651.61...
glasshouse     maxwell      0402.32M037

In [13]:
df1['STATE'].nunique()

98

In [14]:
df1['STATE'].value_counts()

COMPLETED              283333
CANCELLED by 9201       41637
CANCELLED by 174423      6116
RUNNING                  1237
CANCELLED                 947
                        ...  
CANCELLED by 859439         1
CANCELLED by 0              1
CANCELLED by 765671         1
CANCELLED by 856815         1
CANCELLED by 899035         1
Name: STATE, Length: 98, dtype: int64

In [15]:
df1['STATE'].unique()

array(['COMPLETED', 'CANCELLED by 9201', 'CANCELLED by 859023',
       'CANCELLED by 153500', 'CANCELLED by 168618',
       'CANCELLED by 888717', 'FAILED', 'CANCELLED by 859269',
       'CANCELLED by 756088', 'CANCELLED by 887337',
       'CANCELLED by 858683', 'CANCELLED', 'CANCELLED by 885613',
       'CANCELLED by 342288', 'CANCELLED by 855431',
       'CANCELLED by 491383', 'CANCELLED by 776393',
       'CANCELLED by 844796', 'CANCELLED by 780767',
       'CANCELLED by 505355', 'CANCELLED by 649321',
       'CANCELLED by 890189', 'CANCELLED by 59229', 'CANCELLED by 649319',
       'CANCELLED by 590520', 'CANCELLED by 478095',
       'CANCELLED by 196703', 'CANCELLED by 308082',
       'CANCELLED by 757496', 'CANCELLED by 454080',
       'CANCELLED by 512947', 'CANCELLED by 762866',
       'CANCELLED by 859465', 'CANCELLED by 161909',
       'CANCELLED by 785491', 'CANCELLED by 90423', 'CANCELLED by 159541',
       'CANCELLED by 859439', 'CANCELLED by 349259',
       'CANCELLED by 

In [16]:
df2 = df1.groupby(['ACCOUNT','PARTITION','STATE'])['USEDMEM', 'USEDTIME'].agg('sum')
df2.head(10)


C:\Users\upadh\AppData\Local\Temp\ipykernel_16564\3181272860.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2 = df1.groupby(['ACCOUNT','PARTITION','STATE'])['USEDMEM', 'USEDTIME'].agg('sum')


USEDMEM  \
ACCOUNT PARTITION STATE                                                                    
almonds turing    CANCELLED            959.70M2347.09M2377.10M2831.22M01734.94M2106.8...   
                  CANCELLED by 879907                                                 00   
                  CANCELLED by 891323                                           00000000   
                  CANCELLED by 892663                                           00000000   
                  COMPLETED            43427.70M040875.91M40023.96M44535.52M43227.90M...   
                  FAILED                  3142.48M2727.75M2758.29M2506.93M70.55M2681.18M   
antares pascal    CANCELLED                                                       000000   
                  CANCELLED by 349259                                          000000000   
                  CANCELLED by 890189                                               0000   
                  COMPLETED            1475.62M1779.30M1784.97M1784.22M1780.81M1783.6...   

                                                                                USEDTIME  
ACCOUNT PARTITION STATE                                                                   
almonds turing    CANCELLED            00:24:5300:30:5300:30:1200:45:1700:00:0500:05:...  
                  CANCELLED by 879907                                   00:00:0000:00:00  
                  CANCELLED by 891323  00:00:0000:00:0000:00:0000:00:0000:00:0000:00:...  
                  CANCELLED by 892663  00:00:0000:00:0000:00:0000:00:0000:00:0000:00:...  
                  COMPLETED            04:32:3500:00:1700:01:5600:02:1712:02:3405:43:...  
                  FAILED                00:26:3100:16:2800:25:0000:03:4300:02:0300:18:11  
antares pascal    CANCELLED             00:00:1500:00:0200:00:0200:00:0200:00:0200:00:03  
                  CANCELLED by 349259  00:00:0000:00:0000:00:0000:00:0000:00:0000:00:...  
                  CANCELLED by 890189                   00:00:0000:00:0000:00:0000:00:00  
                  COMPLETED            00:10:1700:55:1201:02:0401:10:4501:07:1101:04:...

**making a df with only the completed jobs**

In [17]:
df_State_comleted = df1.loc[df1['STATE'] == 'COMPLETED']

In [18]:
df_State_comleted.shape

(283333, 12)

In [19]:
df_State_comleted['ACCOUNT'].value_counts()

glasshouse       237060
malanga            9901
virginia           9556
prince             7459
grisette           5786
harovin            4729
antares            3203
sunburst           1390
pasilla            1329
plumcot            1027
pecans              609
kinnow              380
bertanne            195
almonds             189
cactus              113
clara                76
orange               74
rosemarie            55
horseradish          48
apple                45
white                31
sugranineteen        31
pinole               19
gem                  19
packham               7
summer                1
bunch                 1
Name: ACCOUNT, dtype: int64

In [20]:
timeInterval ='00:35:01'
list = timeInterval.split(':')
hours = list[0]
minutes = list[1]
seconds = list[2]
total = (int(hours) * 3600 + int(minutes) * 60 + int(seconds))
print("total = ", total)

total =  2101


In [21]:
#df1['Time'] = pd.to_timedelta(df1['USEDTIME']) #this is not needed we will split using the regullr expressions

**Trying to make a function to convert the USEDTIME into a seonds column**

In [22]:
time = df1['USEDTIME'].tolist()[0] # converting the first row into the list and spliting on the :
time_list = time.split(':')
print(time_list)

hours = time_list[0]
minutes = time_list[1]
seconds = time_list[2]
total = (int(hours) * 3600 + int(minutes) * 60 + int(seconds))
print(total)

['00', '13', '13']
793


In [23]:
df1.head(10)

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
5,32880721,glasshouse,brady,403.40M,3-12:00:00,18:49:36,1,1,1,pascal,0:0,COMPLETED
6,32880722,glasshouse,brady,0,3-12:00:00,3-12:00:07,1,1,1,pascal,0:0,COMPLETED
7,32880741,malanga,arline,1119.66M,2-00:00:00,08:09:18,1,2,1,pascal,0:0,COMPLETED
8,32880758,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,CANCELLED by 9201
9,32880759,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,CANCELLED by 9201


In [24]:
time = df1['USEDTIME'].tolist()[6]
time_list = time.split('-',':')
time_list

TypeError: 'str' object cannot be interpreted as an integer

In [25]:
import re

In [26]:
re.split('[-:]', time)

['3', '12', '00', '07']

In [37]:
time = df1['USEDTIME'].tolist()[6]
time_list1 = re.split('[-:]', time)
print(time_list1)
x = []

def time_function(time_list1):         
    days = time_list1[0]
    hours = time_list1[1]
    minutes = time_list1[2]
    seconds = time_list1[3]
    total_1 = (int(days)* 86400 + int(hours) * 3600 + int(minutes) * 60 + int(seconds))
    #x.append(total)
    print(total_1)

['3', '12', '00', '07']


In [39]:
time_list1  = re.split('[-:]', time)
time_function(time_list1)

##it is working for the days in the column (d-hh:mm:ss) we need to modify the slplit arrgument so that the list item at index [0] becomes 0 if there are no days.

302407
